In [1]:
import tensorflow as tf
from utils import *
from config import *
from pytz import timezone
from datetime import datetime
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from import_data import Import_EfficientNetB7_data, Import_TripletNet_train_data
from model.EfficientNetB7 import EfficientNetB7_model
from model.TripletNet import TripletNet_model
from tensorflow.keras.applications.efficientnet import preprocess_input

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
if MODEL_NAME == 'TripletNet':
    train_triplet_generator = Import_TripletNet_train_data(TRAIN_DATA_PATH, BATCH_SIZE, IMAGE_SIZE)
    val_triplet_generator = Import_TripletNet_train_data(VAL_DATA_PATH, BATCH_SIZE, IMAGE_SIZE)
elif MODEL_NAME == 'EfficientNetB7':
    train_generator, val_generator = Import_EfficientNetB7_data(IMAGE_SIZE, BATCH_SIZE, train_data_path=TRAIN_DATA_PATH, val_data_path=VAL_DATA_PATH).build_generators('train')

callbacks = create_callbacks(CHECKPOINT_PATH, CHECKPOINT_FILE_PATH)

In [5]:
if not os.path.exists(RESULT_FILE_PATH):
	os.makedirs(RESULT_FILE_PATH) 

start = datetime.now(timezone('Asia/Seoul'))
print(f"Train start : {start}")

if MODEL_NAME == 'TripletNet':
    if PRETRAINED_CHECKPOINT_PATH:
        print(f"Pretrained checkpoint found: {PRETRAINED_CHECKPOINT_PATH}")
    else:
        print("No pretrained checkpoint found")
    model = TripletNet_model(IMAGE_SIZE, PRETRAINED_CHECKPOINT_PATH).configure_model()
    model.summary()
    model.add_loss(triplet_loss(model.outputs[0], model.outputs[1], model.outputs[2]))
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE))
    history = model.fit(
        train_triplet_generator,
        steps_per_epoch=len(train_triplet_generator),
        validation_data=val_triplet_generator,
        validation_steps=len(val_triplet_generator),
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=0
    )
elif MODEL_NAME == 'EfficientNetB7':
    model = EfficientNetB7_model(IMAGE_SIZE).configure_model()
    model.summary()
    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=LEARNING_RATE),
        metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
    )
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=val_generator,
        validation_steps=val_generator.samples // val_generator.batch_size,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=0
    )

end = datetime.now(timezone('Asia/Seoul'))
print(f"Train end : {end}")

Train start : 2024-06-06 04:40:05.863058+09:00
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb7 (Functional)  (None, 7, 7, 2560)       64097687  
                                                                 
 global_average_pooling2d (G  (None, 2560)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2560)              0         
                                                                 
 dense (Dense)               (None, 1024)              2622464   
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)

c:\Users\USER\anaconda3\envs\capstone\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)



Epoch 00002: val_loss did not improve from 1.98800

Epoch 00003: val_loss did not improve from 1.98800

Epoch 00004: val_loss improved from 1.98800 to 1.80419, saving model to C:/Users/USER/Desktop/Git/capstone/Capstone\train_result\EfficientNetB7_cropped_data_224_1000_0.0003_8\checkpoint\checkpoint-004-1.786765-1.804189-0.282159-0.292157.h5

Epoch 00005: val_loss did not improve from 1.80419

Epoch 00006: val_loss did not improve from 1.80419


In [ ]:
save_result(history)